# Problem

    In energy production, the power grid is a a large directed graph of energy consumers and producers. At times you need to cut at certain nodes and trim demand because you cannot supply enough of a load.

    In DailyProgrammeropolis, all buildings are connected to the grid and all consume power to varying degrees. Some generate power because they have installed on-site generation and sell the excess to the grid, some do not.

    The scenario you're facing is this: due to a fault with the bulk power generation facility not local to DailyProgrammerololis, you must trim the power grid. You have connectivity data, and power consumption and production data. Your goal with this challenge is to maximize the number of powered nodes with the generated energy you have. Note that when you cut off a node, you run the risk the downstream ones will loose power, too, if they are no longer connected. This is how you'll shed demand, by selectively cutting the graph. You can make as many cuts as you want (there is no restriction on this).
   
# Input

    You'll be given an extensive set of data for this challenge. The first set of data looks like this: you'll be given a single integer on one line telling you how many nodes to read. Then you'll be given those nodes, one per line, with the node ID, the amount of power it consumes in kWH, then how much the node generates in kWH. Not all nodes produce electricity, but some do (e.g. a wind farm, solar cells, etc), and there is obviously one that generates the most - that's your main power plant.
    
    The next set of data is the edge data. The first line is how many edges to read, then the next N lines have data showing how the nodes are connected (e.g. power flows from node a to b).
    
# Output

    Your program should emit a list of edges to sever as a list of (i,j) two tuples. Multiple answers are possible. You may wind up with a number of small islands as opposed to one powered network.

# Pseudo

    This graph can be approached as a directed gaph.
    For each edge in microgrid_edges, create 2 directed edges.
    Each has weight equal to tail of directed edge.
    
    Search for a Minimum Spanning Arborescence (MSA) on this graph.
    
    To allow for islands, 
    we add a dummy node that connects all nodes with net power generation > 0.
    
    Then we start removing leaves with max weight, untill the total power consumption <= production.
    
    As each edge of this modified MSA has a corresponding edge in the original graph,
    we can use this solution for the undirected graph.
    
    I am not certain that this solution is optimal.

# Nodes and edges

In [205]:
with open('nodes.txt') as file:
    space = [line for line in file.read().split('\n')]

size_n = int(space[0])
l_nodes = [[int(float(x)*10**3) for x in line.split(' ')] for line in space[1:]]
d_nodes = {x//10**3:(z-y) for x,y,z in l_nodes}

In [206]:
with open('microgrid_edges.txt') as file:
    space = [line for line in file.read().split('\n')]
    
size_e = int(space[0])
l_edges = [[int(x) for x in line.split(' ')] for line in space[1:-1]]
d_edges = {x:set() for x in range(size_n)}
for x,y in l_edges:
    d_edges[x].add(y)
    d_edges[y].add(x)   

# Dummy Node

In [207]:
d_edges[-1] = set()
for x,y in d_nodes.items():
    if y >= 0:
        d_edges[-1].add(x)
        d_edges[x].add(-1)
d_nodes[-1] = 0

# Minimum spanning arborescence

- [Wiki](https://en.wikipedia.org/wiki/Minimum_spanning_tree) about minimum spanning tree

- [Wiki](https://en.wikipedia.org/wiki/Arborescence_(graph_theory) on arborescence

- [Stackoverflow](https://stackoverflow.com/questions/23988236/chu-liu-edmonds-algorithm-for-minimum-spanning-tree-on-directed-graphs) code source


In [208]:
#!/usr/bin/env python3
from collections import defaultdict, namedtuple

Arc = namedtuple('Arc', ('tail', 'weight', 'head'))

def min_spanning_arborescence(arcs, sink):
    good_arcs = []
    quotient_map = {arc.tail: arc.tail for arc in arcs}
    quotient_map[sink] = sink
    while True:
        min_arc_by_tail_rep = {}
        successor_rep = {}
        for arc in arcs:
            if arc.tail == sink:
                continue
            tail_rep = quotient_map[arc.tail]
            head_rep = quotient_map[arc.head]
            if tail_rep == head_rep:
                continue
            if tail_rep not in min_arc_by_tail_rep or min_arc_by_tail_rep[tail_rep].weight > arc.weight:
                min_arc_by_tail_rep[tail_rep] = arc
                successor_rep[tail_rep] = head_rep
        cycle_reps = find_cycle(successor_rep, sink)
        if cycle_reps is None:
            good_arcs.extend(min_arc_by_tail_rep.values())
            return spanning_arborescence(good_arcs, sink)
        good_arcs.extend(min_arc_by_tail_rep[cycle_rep] for cycle_rep in cycle_reps)
        cycle_rep_set = set(cycle_reps)
        cycle_rep = cycle_rep_set.pop()
        quotient_map = {node: cycle_rep if node_rep in cycle_rep_set else node_rep for node, node_rep in quotient_map.items()}


def find_cycle(successor, sink):
    visited = {sink}
    for node in successor:
        cycle = []
        while node not in visited:
            visited.add(node)
            cycle.append(node)
            node = successor[node]
        if node in cycle:
            return cycle[cycle.index(node):]
    return None


def spanning_arborescence(arcs, sink):
    arcs_by_head = defaultdict(list)
    for arc in arcs:
        if arc.tail == sink:
            continue
        arcs_by_head[arc.head].append(arc)
    solution_arc_by_tail = {}
    stack = arcs_by_head[sink]
    while stack:
        arc = stack.pop()
        if arc.tail in solution_arc_by_tail:
            continue
        solution_arc_by_tail[arc.tail] = arc
        stack.extend(arcs_by_head[arc.tail])
    return solution_arc_by_tail

# min_spanning_arborescence([Arc(1, 17, 0),  Arc(2, 16, 0),  Arc(3, 19, 0),  Arc(4, 16, 0),  Arc(5, 16, 0),  Arc(6, 18, 0),  Arc(2, 3, 1),  Arc(3, 3, 1),  Arc(4, 11, 1),  Arc(5, 10, 1),  Arc(6, 12, 1),  Arc(1, 3, 2),  Arc(3, 4, 2),  Arc(4, 8, 2),  Arc(5, 8, 2),  Arc(6, 11, 2),  Arc(1, 3, 3),  Arc(2, 4, 3),  Arc(4, 12, 3),  Arc(5, 11, 3),  Arc(6, 14, 3),  Arc(1, 11, 4),  Arc(2, 8, 4),  Arc(3, 12, 4),  Arc(5, 6, 4),  Arc(6, 10, 4),  Arc(1, 10, 5),  Arc(2, 8, 5),  Arc(3, 11, 5),  Arc(4, 6, 5),  Arc(6, 4, 5),  Arc(1, 12, 6),  Arc(2, 11, 6),  Arc(3, 14, 6),  Arc(4, 10, 6),  Arc(5, 4, 6)], 0)

We use the big power hub, at node 100, as the source for the MSA.

In [209]:
Arcs = []

for x,edges in d_edges.items():
    for edge in edges:        
        Arcs.append(Arc(x,-d_nodes[edge],edge))

MSA = min_spanning_arborescence(Arcs,100)

In [210]:
MSA_edges = {x:[set(),set(),y] for x,y in d_nodes.items()}

for tail,weight,head in MSA.values():
    MSA_edges[tail][0].add(head) 
    MSA_edges[head][1].add(tail)    

In [211]:
import heapq

total = sum(x[2] for x in MSA_edges.values())

while total < 0:
    heap = []
    for x,y in MSA_edges.items():
        if not y[1]:
            heapq.heappush(heap,(y[2],x))
    change, node = heapq.heappop(heap)
    total -= change
    MSA_edges[MSA_edges[node][0].pop()][1].discard(node)
    MSA_edges.pop(node)    

In [212]:
MSA_edges.pop(-1)
len(MSA_edges),sum(x[2] for x in MSA_edges.values())

(88, 1597)

In [213]:
MSA_edges.keys()

dict_keys([0, 2, 4, 5, 6, 8, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 97, 98, 99, 100])

# Solution

    Your program should emit a list of edges to sever as a list of (i,j) two tuples. 
    Multiple answers are possible. 
    You may wind up with a number of small islands as opposed to one powered network.
      
 *You can make as many cuts as you want*
 
    Rather than returning a list of all to removen, i return a list of all edges that should not be broken.

In [219]:
output = [(x,*y[0]) for x,y in MSA_edges.items() if y[0]]
output

[(0, 34),
 (2, 81),
 (4, 61),
 (5, 84),
 (6, 95),
 (8, 36),
 (10, 54),
 (11, 54),
 (12, 36),
 (13, 2),
 (14, 55),
 (16, 5),
 (17, 78),
 (18, 100),
 (19, 95),
 (20, 5),
 (21, 100),
 (22, 20),
 (23, 91),
 (24, 64),
 (25, 94),
 (28, 84),
 (29, 91),
 (30, 34),
 (31, 77),
 (32, 64),
 (33, 10),
 (34, -1),
 (35, 61),
 (36, 100),
 (37, 65),
 (38, 22),
 (39, 84),
 (40, 100),
 (41, 91),
 (42, 95),
 (43, 2),
 (44, 80),
 (45, 78),
 (46, 61),
 (50, 74),
 (51, 84),
 (52, 5),
 (53, 5),
 (54, 20),
 (55, 32),
 (56, 55),
 (57, 61),
 (58, 61),
 (59, 36),
 (61, 34),
 (62, 84),
 (63, 55),
 (64, 90),
 (65, 61),
 (66, 90),
 (67, 34),
 (68, 91),
 (69, 36),
 (70, 82),
 (71, 84),
 (72, 55),
 (73, 84),
 (74, 73),
 (76, 100),
 (77, 100),
 (78, 64),
 (79, 2),
 (80, 78),
 (81, 68),
 (82, -1),
 (83, 82),
 (84, -1),
 (85, 55),
 (86, 62),
 (87, 2),
 (88, 91),
 (89, 100),
 (90, 100),
 (91, 100),
 (92, 61),
 (93, 91),
 (94, 90),
 (95, 69),
 (97, 62),
 (98, 36),
 (99, 32)]